In [ ]:
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets, manifold
import torch
import math

Path_pictures = f"/home/alazarev/CodeProjects/Experiments/swissroll_embeddings"

alpha = 0.5 # point opacity

In [ ]:
n_samples=18000
sr_points, sr_color = datasets.make_swiss_roll(n_samples=n_samples, noise=0.05,random_state=1)

In [ ]:
points_tensor = torch.tensor(sr_points)
cov_matrix = torch.cov(points_tensor.T)
print("Covariance matrix:\n", cov_matrix)
mean = points_tensor.mean(dim=0)
print("Mean vector:", mean)

In [ ]:
import numpy as np
N = 1000
noise = 0.05

t = 3*np.pi/2 * (1 + 2*np.random.rand(1,N))
h = 21 * np.random.rand(1,N)
data = np.concatenate((t*np.cos(t),h,t*np.sin(t))) + noise*np.random.randn(3,N)	
#np.transpose(data), np.squeeze(t)

In [ ]:
norms = torch.tensor(sr_points).norm(dim = 1)

In [ ]:
min = norms.min()

In [ ]:
max= norms.max()

In [ ]:
mean = norms.mean()
std = norms.std()

In [ ]:
plt.hist(norms)
plt.title("Histogram of norms of all points of the swiss roll")
plt.show()
print(f"Norms vary between {min:0.2f} and {max:0.2f} \n with mean: {mean:0.2f} and std dev: {std:0.2f} ")

In [ ]:
plt.title("Histogram of norms of normalized points of the swiss roll")
plt.hist((norms-mean)/std)
plt.show()

## resampling for plots

In [ ]:
sr_points, sr_color = datasets.make_swiss_roll(n_samples=round(n_samples*0.2), noise=0.05,random_state=1)

In [ ]:
fig = plt.figure(figsize=(9,9),dpi=400)
plt.rcParams.update({'font.size': 20}) # makes all fonts on the plot be 20

ax = fig.add_subplot(projection='3d')
p = ax.scatter(sr_points[:,0],
           sr_points[:,1],
           sr_points[:,2],
           c=sr_color, s=40, alpha = alpha,cmap='jet')
#plt.title("Swiss roll dataset in 3d",fontsize=20)
#plt.colorbar(p, label="sr_color", shrink = 0.5,location="left")
ax.view_init(azim=-70, elev=15)

ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')


ax.grid(True)

#plt.savefig(f'{Path_pictures}/swissroll3d_cbar.pdf',bbox_inches='tight',format='pdf')
#plt.savefig(f'{Path_pictures}/swissroll3d.pdf',bbox_inches='tight',format='pdf')
plt.show()

### The solution of unrolling the doe

In [ ]:
colors_tensor = torch.from_numpy(sr_color)
#colors_normalized = torch.nn.functional.normalize(colors_tensor,dim=0)
y_tensor = torch.from_numpy(sr_points[:,1])
#y_normalized = torch.nn.functional.normalize(y_tensor,dim=0)
fig = plt.figure(figsize=(9,9),dpi=400)
plt.rcParams.update({'font.size': 20})
#plt.figure(figsize=(20,10),dpi=100)
#plt.title("The unrolled swiss roll: projection on the\n 2nd and the 4th coordinate (the color)")
#plt.xlabel("the y-coordinate of sr_points")
#plt.ylabel("sr_color")
#plt.scatter(y_normalized,colors_normalized,c=sr_color)
plt.scatter(y_tensor,colors_tensor,c=sr_color,cmap='jet',s=40,alpha=alpha)
#plt.savefig(f'{Path_pictures}/swissroll_solution.pdf',bbox_inches='tight',format='pdf')
plt.show()

### LLE and t-SNE

In [ ]:
sr_lle, sr_err = manifold.locally_linear_embedding(
    sr_points, n_neighbors=12, n_components=2
)

sr_tsne = manifold.TSNE(n_components=2, perplexity=40, random_state=0).fit_transform(
    sr_points
)
"""
fig, axs = plt.subplots(figsize=(8, 8), nrows=2)
axs[0].scatter(sr_lle[:, 0], sr_lle[:, 1], c=sr_color)
axs[0].set_title("LLE Embedding of Swiss Roll")
axs[1].scatter(sr_tsne[:, 0], sr_tsne[:, 1], c=sr_color)
_ = axs[1].set_title("t-SNE Embedding of Swiss Roll")
"""


In [ ]:
plt.rcParams.update({'font.size': 20})
fig = plt.figure(figsize=(9,9),dpi=400)
plt.scatter(sr_lle[:, 0], sr_lle[:, 1], c=sr_color,cmap='jet',s=40,alpha=alpha)
#plt.title("LLE Embedding of the swiss roll")
#plt.savefig(f'{Path_pictures}/swissroll_lle.pdf',bbox_inches='tight',format='pdf')
plt.show()

In [ ]:
fig = plt.figure(figsize=(9,9),dpi=400)
plt.rcParams.update({'font.size': 20})
plt.scatter(sr_tsne[:, 0], sr_tsne[:, 1], c=sr_color,cmap='jet',s=40,alpha=alpha)
#plt.title("t-SNE embedding of the swiss roll")
#plt.savefig(f'{Path_pictures}/swissroll_tsne.pdf',bbox_inches='tight',format='pdf')
#plt.savefig(f'{Path_pictures}/swissroll_tsne.pdf',bbox_inches='tight',format='pdf')
plt.show()

### UMAP

In [ ]:
import umap

In [ ]:
mapper = umap.UMAP().fit(sr_points)

In [ ]:
encoded_points = mapper.embedding_

In [ ]:
fig = plt.figure(figsize=(9,9),dpi=400)
plt.rcParams.update({'font.size': 20}) # makes all fonts on the plot be 20
plt.scatter( encoded_points[:,0], encoded_points[:,1], c=sr_color, s= 40,alpha=alpha, cmap='jet',marker='o',edgecolors=None )
#plt.title( "UMAP embedding of the swiss roll")
#plt.colorbar(orientation='vertical',shrink = 0.7)
#plt.savefig(f'{Path_pictures}/swissroll_umap.pdf',bbox_inches='tight',format='pdf')
plt.show()

# PCA

In [ ]:
u,s,v = torch.pca_lowrank(torch.tensor(sr_points),q=2)

In [ ]:
plt.figure(figsize=(9,9),dpi=400)
plt.rcParams.update({'font.size': 20}) # makes all fonts on the plot be 20
plt.scatter( u[:,0], u[:,1], c=sr_color, s= 40,alpha=alpha, cmap='jet',marker='o',edgecolors=None )
#plt.title( "PCA embedding of the swiss roll")
#plt.colorbar(orientation='vertical',shrink = 0.7)
#plt.savefig(f'{Path_pictures}/swissroll_pca.pdf',bbox_inches='tight',format='pdf')
plt.show()

# AE

In [ ]:
# Minimal imports
import math
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# adding path to the set generating package
import sys
sys.path.append('../') # have to go 1 level up
import ricci_regularization as RR
#import torchvision


import sklearn
from sklearn import datasets

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, 128)
        self.linear4 = nn.Linear(128, hidden_dim)
        #self.activation = nn.ReLU()
        self.activation = torch.sin
    def forward(self, x):
        y = self.linear1(x)
        y = self.activation(y)
        y = self.linear2(y)
        y = self.activation(y)
        y = self.linear3(y)
        y = self.activation(y)
        out = self.linear4(y)
        #out = self.activation(out)
        return out

In [ ]:
load_weight_name = "swissroll_curv_w=0_ls=R^2"
encoder = Encoder(input_dim=3, hidden_dim=2)
PATH_enc = f'../nn_weights/encoder_{load_weight_name}'
encoder.load_state_dict(torch.load(PATH_enc))
encoder.eval()

In [ ]:
sr_points_tensor = torch.tensor(sr_points).to(dtype=torch.float32)
sr_in_AE_ls = encoder(sr_points_tensor).detach()

In [ ]:
fig = plt.figure(figsize=(9,9),dpi=400)
plt.rcParams.update({'font.size': 20})
plt.scatter(sr_in_AE_ls[:, 0], sr_in_AE_ls[:, 1], 
            c=sr_color,cmap='jet',s=40,alpha=alpha)
#plt.title("t-SNE embedding of the swiss roll")
#plt.savefig(f'{Path_pictures}/swissroll_tsne.png',bbox_inches='tight',format='png')
plt.savefig(f'{Path_pictures}/swissroll_AE.pdf',bbox_inches='tight',format='pdf')
plt.show()